#1. Prepare Python that loads naics4 data into Pandas for 2017 to 2021 for Maine
Source files. Load these directly from the URL into Pandas.

https://model.earth/community-data/industries/naics/US/counties/ME/US-ME-census-naics4-counties-2021.csv

View files on GitHub: industries/naics/US/counties/ME

Add your first name instead of "TO DO" if you are working on one of these:  

Fanyi: Change to a range of years: 2017 to 2021  
Example: https://model.earth/data-pipeline/industries/naics  

Fanyi: Add a "SqKm" column after the population column.  
Advice on python libraries from ChatGPT  
https://chat.openai.com/share/19f29b3a-01f5-4620-b930-26394c4e25cf

Sijia: The UN defines 1,500 or more people per square kilometer (km²) as an urban area. Add an "UrbanPercent" column with .5 = 50% when half the land area is 1500 or more. Set to a max of 1 when all the urban areas (within a county) have populations >= 1,500 people/km².

TO DO: Add an "UrbanDensity" column with the average urban population for all Urban areas in the county (where population is over 1,500 people/km²) as a factor with 1=100% with 1=1500, 2=3,000. Round to 1 decimal, so a population of 1,900 people/km² would be 1900/1500 = 1.266 rounded to 1.3.

In [ ]:
import pandas as pd
import requests
from io import StringIO

In [ ]:
year=2021
state="ME"
sample_url = f"https://model.earth/community-data/industries/naics/US/counties/{state}/US-{state}-census-naics4-counties-{year}.csv"

In [ ]:
def get_df(url):
    response = requests.get(url)
    df = pd.read_csv(StringIO(response.text))
    return df

In [ ]:
df=get_df(sample_url)
df

,Fips,Naics,Establishments,Employees,Payroll
0,23001,1133,10,47,2463
1,23001,1152,5,4,103
2,23001,2211,12,130,12254
3,23001,2361,85,229,9476
4,23001,2362,14,127,7877
...,...,...,...,...,...
1829,23029,8129,4,13,240
1830,23029,8131,14,34,389
1831,23029,8133,8,27,1506
1832,23029,8134,3,43,2049


In [ ]:
len(df.Naics.unique())

223

#2. Output annual training files to
output/2021/2021-naics2-counties.csv

To streamline reuse, we'll avoid creating a header row.

Each row is a location (Fips = countyID) with 73 industries, each with 3 attributes (224 columns total)

Industry's Establishments per 1000 people
Industry's Employees per 1000 people
Industry's Average pay per Employee

We'll need a Python library that pulls the county population by year.
Let's also include the value of the center latitude and longitude
We could include the county name, then see if the model predicts differently without it.

A file called columns.md could be output with a list of the column values:

Fips
Name
Population
Latitude
Longitude
Naics X Establishments per 1000
Naics X Employees per 1000
Naics X Average pay
[repeat]

As a reference, the prior structure for ztca (zipcode) was:

zcta, JobsTotal, JobsAgriculture, JobsEntertainment,…
Population, Poverty, PovertyUnder18, Education,WorkExperience, WorkingFulltime, WorkingFulltime_Poverty, y

Ronan is working on a similar pivot in prep/industries for timelines, except the timeline rows are years.

## pivot

In [ ]:
df=df.pivot_table(index="Fips", columns='Naics', values=["Establishments","Employees","Payroll"]).reset_index()
df.columns = [f"{a}_{str(b)}" for a,b in df.columns]
sorted_columns = ["Fips_"]+sorted(df.columns[1:], key=lambda x: int(x.split('_')[-1]))
df = df[sorted_columns].rename(columns={"Fips_":"Fips"})

In [ ]:
df

,Fips,Employees_1131,Establishments_1131,Payroll_1131,Employees_1133,Establishments_1133,Payroll_1133,Employees_1141,Establishments_1141,Payroll_1141,...,Payroll_8132,Employees_8133,Establishments_8133,Payroll_8133,Employees_8134,Establishments_8134,Payroll_8134,Employees_8139,Establishments_8139,Payroll_8139
0,23001,NaN,NaN,NaN,47.0,10.0,2463.0,NaN,NaN,NaN,...,NaN,40.0,9.0,1678.0,155.0,24.0,2647.0,21.0,8.0,1296.0
1,23003,NaN,NaN,NaN,521.0,133.0,22736.0,NaN,NaN,NaN,...,223.0,20.0,3.0,901.0,22.0,8.0,270.0,76.0,11.0,1648.0
2,23005,NaN,NaN,NaN,30.0,13.0,1266.0,43.0,69.0,2759.0,...,20536.0,443.0,68.0,29132.0,345.0,64.0,11040.0,426.0,84.0,20425.0
3,23007,NaN,NaN,NaN,138.0,21.0,6141.0,NaN,NaN,NaN,...,257.0,18.0,8.0,1137.0,12.0,4.0,142.0,8.0,5.0,229.0
4,23009,NaN,NaN,NaN,21.0,6.0,976.0,136.0,139.0,8391.0,...,5355.0,72.0,15.0,3728.0,49.0,9.0,1327.0,52.0,18.0,1502.0
5,23011,NaN,NaN,NaN,55.0,11.0,3345.0,NaN,NaN,NaN,...,7048.0,211.0,29.0,10376.0,89.0,24.0,2385.0,346.0,64.0,25263.0
6,23013,NaN,NaN,NaN,27.0,5.0,1250.0,422.0,145.0,30792.0,...,831.0,115.0,17.0,5637.0,50.0,8.0,1365.0,15.0,9.0,806.0
7,23015,NaN,NaN,NaN,9.0,4.0,413.0,95.0,83.0,4773.0,...,1022.0,36.0,5.0,1896.0,67.0,7.0,1988.0,10.0,7.0,434.0
8,23017,NaN,NaN,NaN,158.0,42.0,8898.0,NaN,NaN,NaN,...,282.0,16.0,8.0,541.0,15.0,7.0,308.0,61.0,13.0,1333.0
9,23019,30.0,3.0,2134.0,496.0,52.0,23276.0,NaN,NaN,NaN,...,6578.0,82.0,18.0,3331.0,106.0,19.0,2750.0,52.0,16.0,2150.0


## population, name, long,lat

#### censusdata

In [ ]:
!pip install censusdata

In [ ]:
def get_name_population_cencusdata(year,fips):
  import censusdata
  state_fips_code = str(fips)[0:2]
  county_fips_code= str(fips)[-3:]
  data = pd.DataFrame(censusdata.download('acs5', year, censusdata.censusgeo([('state', state_fips_code), ('county',county_fips_code )]), ['B01003_001E']))
  data=data.reset_index().rename(columns={'index': 'county',"B01003_001E":"total_population"})
  data.county=data.county.apply(lambda x: str(x).split()[0])
  return data.county[0],data.total_population[0]

#### cenpy

In [ ]:
!pip install cenpy

In [ ]:
def get_name_population_cenpy(year,fips):
    import cenpy
    state_code=str(fips)[0:2]
    county_code=str(fips)[-3:]
    # Set the connection to the Census API for the specific year
    conn = cenpy.products.APIConnection(f"ACSDT5Y{year}")
    # Get the county population data for all counties in the US for the current year
    df = conn.query(cols=["NAME", "B01001_001E"], geo_filter={"state":state_code},geo_unit=f"county:{county_code}")
    name=df.NAME[0].split(",")[0]
    pop=df.B01001_001E[0]
    return name,pop

### add name and population

In [ ]:
df

,Fips,Employees_1131,Establishments_1131,Payroll_1131,Employees_1133,Establishments_1133,Payroll_1133,Employees_1141,Establishments_1141,Payroll_1141,...,Payroll_8132,Employees_8133,Establishments_8133,Payroll_8133,Employees_8134,Establishments_8134,Payroll_8134,Employees_8139,Establishments_8139,Payroll_8139
0,23001,NaN,NaN,NaN,47.0,10.0,2463.0,NaN,NaN,NaN,...,NaN,40.0,9.0,1678.0,155.0,24.0,2647.0,21.0,8.0,1296.0
1,23003,NaN,NaN,NaN,521.0,133.0,22736.0,NaN,NaN,NaN,...,223.0,20.0,3.0,901.0,22.0,8.0,270.0,76.0,11.0,1648.0
2,23005,NaN,NaN,NaN,30.0,13.0,1266.0,43.0,69.0,2759.0,...,20536.0,443.0,68.0,29132.0,345.0,64.0,11040.0,426.0,84.0,20425.0
3,23007,NaN,NaN,NaN,138.0,21.0,6141.0,NaN,NaN,NaN,...,257.0,18.0,8.0,1137.0,12.0,4.0,142.0,8.0,5.0,229.0
4,23009,NaN,NaN,NaN,21.0,6.0,976.0,136.0,139.0,8391.0,...,5355.0,72.0,15.0,3728.0,49.0,9.0,1327.0,52.0,18.0,1502.0
5,23011,NaN,NaN,NaN,55.0,11.0,3345.0,NaN,NaN,NaN,...,7048.0,211.0,29.0,10376.0,89.0,24.0,2385.0,346.0,64.0,25263.0
6,23013,NaN,NaN,NaN,27.0,5.0,1250.0,422.0,145.0,30792.0,...,831.0,115.0,17.0,5637.0,50.0,8.0,1365.0,15.0,9.0,806.0
7,23015,NaN,NaN,NaN,9.0,4.0,413.0,95.0,83.0,4773.0,...,1022.0,36.0,5.0,1896.0,67.0,7.0,1988.0,10.0,7.0,434.0
8,23017,NaN,NaN,NaN,158.0,42.0,8898.0,NaN,NaN,NaN,...,282.0,16.0,8.0,541.0,15.0,7.0,308.0,61.0,13.0,1333.0
9,23019,30.0,3.0,2134.0,496.0,52.0,23276.0,NaN,NaN,NaN,...,6578.0,82.0,18.0,3331.0,106.0,19.0,2750.0,52.0,16.0,2150.0


In [ ]:
df.insert(loc=1, column='name', value=df.Fips.apply(lambda x:get_name_population_cenpy(year,x)[0]))
df.insert(loc=2, column='population', value=df.Fips.apply(lambda x:get_name_population_cenpy(year,x)[1]))

In [ ]:
df

,Fips,name,population,Employees_1131,Establishments_1131,Payroll_1131,Employees_1133,Establishments_1133,Payroll_1133,Employees_1141,...,Payroll_8132,Employees_8133,Establishments_8133,Payroll_8133,Employees_8134,Establishments_8134,Payroll_8134,Employees_8139,Establishments_8139,Payroll_8139
0,23001,Androscoggin County,110378,NaN,NaN,NaN,47.0,10.0,2463.0,NaN,...,NaN,40.0,9.0,1678.0,155.0,24.0,2647.0,21.0,8.0,1296.0
1,23003,Aroostook County,67272,NaN,NaN,NaN,521.0,133.0,22736.0,NaN,...,223.0,20.0,3.0,901.0,22.0,8.0,270.0,76.0,11.0,1648.0
2,23005,Cumberland County,300776,NaN,NaN,NaN,30.0,13.0,1266.0,43.0,...,20536.0,443.0,68.0,29132.0,345.0,64.0,11040.0,426.0,84.0,20425.0
3,23007,Franklin County,29487,NaN,NaN,NaN,138.0,21.0,6141.0,NaN,...,257.0,18.0,8.0,1137.0,12.0,4.0,142.0,8.0,5.0,229.0
4,23009,Hancock County,55417,NaN,NaN,NaN,21.0,6.0,976.0,136.0,...,5355.0,72.0,15.0,3728.0,49.0,9.0,1327.0,52.0,18.0,1502.0
5,23011,Kennebec County,123293,NaN,NaN,NaN,55.0,11.0,3345.0,NaN,...,7048.0,211.0,29.0,10376.0,89.0,24.0,2385.0,346.0,64.0,25263.0
6,23013,Knox County,40519,NaN,NaN,NaN,27.0,5.0,1250.0,422.0,...,831.0,115.0,17.0,5637.0,50.0,8.0,1365.0,15.0,9.0,806.0
7,23015,Lincoln County,35065,NaN,NaN,NaN,9.0,4.0,413.0,95.0,...,1022.0,36.0,5.0,1896.0,67.0,7.0,1988.0,10.0,7.0,434.0
8,23017,Oxford County,57807,NaN,NaN,NaN,158.0,42.0,8898.0,NaN,...,282.0,16.0,8.0,541.0,15.0,7.0,308.0,61.0,13.0,1333.0
9,23019,Penobscot County,152211,30.0,3.0,2134.0,496.0,52.0,23276.0,NaN,...,6578.0,82.0,18.0,3331.0,106.0,19.0,2750.0,52.0,16.0,2150.0


### long&lat



In [ ]:
from geopy.geocoders import Nominatim
# Initialize Nominatim geocoder
def get_long_lat(name):
  geolocator = Nominatim(user_agent="my_geocoder")
  county_coordinates = {} # Pulls the km
  location = geolocator.geocode(name)
  lat=location.latitude
  lon=location.longitude
  return lon,lat

#### add long&lat

In [ ]:
df.insert(loc=3, column='longitude', value=df.name.apply(lambda x:get_long_lat(x)[0]))
df.insert(loc=4, column='latitude', value=df.name.apply(lambda x:get_long_lat(x)[1]))

In [ ]:
df

,Fips,name,population,longitude,latitude,Employees_1131,Establishments_1131,Payroll_1131,Employees_1133,Establishments_1133,...,Payroll_8132,Employees_8133,Establishments_8133,Payroll_8133,Employees_8134,Establishments_8134,Payroll_8134,Employees_8139,Establishments_8139,Payroll_8139
0,23001,Androscoggin County,110378,-70.196395,44.140284,NaN,NaN,NaN,47.0,10.0,...,NaN,40.0,9.0,1678.0,155.0,24.0,2647.0,21.0,8.0,1296.0
1,23003,Aroostook County,67272,-68.470113,46.684796,NaN,NaN,NaN,521.0,133.0,...,223.0,20.0,3.0,901.0,22.0,8.0,270.0,76.0,11.0,1648.0
2,23005,Cumberland County,300776,-77.280393,40.154158,NaN,NaN,NaN,30.0,13.0,...,20536.0,443.0,68.0,29132.0,345.0,64.0,11040.0,426.0,84.0,20425.0
3,23007,Franklin County,29487,-95.223175,33.200266,NaN,NaN,NaN,138.0,21.0,...,257.0,18.0,8.0,1137.0,12.0,4.0,142.0,8.0,5.0,229.0
4,23009,Hancock County,55417,-91.145806,40.394171,NaN,NaN,NaN,21.0,6.0,...,5355.0,72.0,15.0,3728.0,49.0,9.0,1327.0,52.0,18.0,1502.0
5,23011,Kennebec County,123293,-69.825066,44.418462,NaN,NaN,NaN,55.0,11.0,...,7048.0,211.0,29.0,10376.0,89.0,24.0,2385.0,346.0,64.0,25263.0
6,23013,Knox County,40519,-99.707362,33.587139,NaN,NaN,NaN,27.0,5.0,...,831.0,115.0,17.0,5637.0,50.0,8.0,1365.0,15.0,9.0,806.0
7,23015,Lincoln County,35065,-81.206272,35.486642,NaN,NaN,NaN,9.0,4.0,...,1022.0,36.0,5.0,1896.0,67.0,7.0,1988.0,10.0,7.0,434.0
8,23017,Oxford County,57807,-70.778238,44.469453,NaN,NaN,NaN,158.0,42.0,...,282.0,16.0,8.0,541.0,15.0,7.0,308.0,61.0,13.0,1333.0
9,23019,Penobscot County,152211,-68.595777,45.293017,30.0,3.0,2134.0,496.0,52.0,...,6578.0,82.0,18.0,3331.0,106.0,19.0,2750.0,52.0,16.0,2150.0


In [ ]:
def get_area(state_fp):
    import geopandas as gpd
    test = gpd.read_file("tl_2017_us_county/tl_2017_us_county.shp")
    tost = test.copy()
    tost= tost.to_crs({'init': 'epsg:3035'})
    tost["area"] = tost['geometry'].area/ 10**6
    tost=tost[tost.STATEFP==state_fp]
    tost["Fips"]=tost.STATEFP+tost.COUNTYFP
    return tost[["Fips","area"]]

In [ ]:
def main(year,state):
    sample_url = f"https://model.earth/community-data/industries/naics/US/counties/{state}/US-{state}-census-naics4-counties-{year}.csv"
    df=get_df(sample_url)
    df=df.pivot_table(index="Fips", columns='Naics', values=["Establishments","Employees","Payroll"]).reset_index()
    df.columns = [f"{a}-{str(b)}" for a,b in df.columns]
    sorted_columns = ["Fips-"]+sorted(df.columns[1:], key=lambda x: int(x.split('-')[-1]))
    df = df[sorted_columns].rename(columns={"Fips-":"Fips"})
    df.insert(loc=1, column='name', value=df.Fips.apply(lambda x:get_name_population_cencusdata(year,x)[0]))
    df.insert(loc=2, column='population', value=df.Fips.apply(lambda x:get_name_population_cencusdata(year,x)[1]))
    df.insert(loc=3, column='longitude', value=df.name.apply(lambda x:get_long_lat(x)[0]))
    df.insert(loc=4, column='latitude', value=df.name.apply(lambda x:get_long_lat(x)[1]))
    state_code=str(df.Fips[0])[0:2]
    area_df=get_area(state_code)
    df.Fips=df.Fips.astype(str)
    df=df.merge(area_df,how="left",on="Fips")
    output_dir = f"../output/{year}"
    os.makedirs(output_dir, exist_ok=True)
    path = f"{output_dir}/US-{state}-training-naics4-counties-{year}.csv"
    df.to_csv(path, header=True, index=False)

In [ ]:
year_range=range(2017,2022)
state_list=['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
for i in year_range:
    for j in state_list:
        try:
            main(i,j)
            print(f"{year}{state} has been saved")
        except:
            continue

ERROR:fiona._env:tl_2017_us_county/tl_2017_us_county.shp: No such file or directory
ERROR:fiona._env:tl_2017_us_county/tl_2017_us_county.shp: No such file or directory


In [ ]:
df.insert(loc=5, column='urban_density', value=df.apply(lambda x:int(x.population)/get_county_area(x.longitude,x.latitude),axis=1))

In [ ]:
def get_urban_percent(population, area):
  population_per_km2 = population/area
  if population_per_km2 >= 1500:
    return 1
  return population_per_km2/1500

In [ ]:
df.insert(loc=6, column='urban_percent', value=df.apply(lambda x:get_urban_percent(int(x.population), x.area),axis=1))

In [ ]:
df

## output

In [ ]:
import os
output_dir = f"../output/{year}"
os.makedirs(output_dir, exist_ok=True)
path = f"{output_dir}/US-{state}-training-naics4-counties-{year}.csv"
df.to_csv(path, header=True, index=False)